# Part 1 initial notebook

Note that in its initial form the notebook will not run. You will need to amend two cells in order to successfully run the notebook. See `assessment.ipynb` for instructions.

In [ ]:
import numpy as np
%matplotlib inline

from examples.seismic import Model, AcquisitionGeometry, Receiver, plot_velocity, plot_image
from examples.seismic.acoustic import AcousticWaveSolver

import matplotlib.pyplot as plt

from devito import Function, TimeFunction, Eq, Operator, norm, mmax, configuration

configuration['log-level'] = 'WARNING'

In [ ]:
nshots = 9  # Number of shots to create gradient from
nreceivers = 101  # Number of receiver locations per shot
fwi_iterations = 5  # Number of outer FWI iterations

In [ ]:
shape = (101, 101)  # Number of grid point (nx, nz)
spacing = (10., 10.)  # Grid spacing in m. The domain size is now 1km by 1km
origin = (0., 0.)  # Need origin to define relative source and receiver locations

In [ ]:
vback = 2.0
v0 = np.full(shape, vback)

plot_image(v0, cmap="seismic")

In [ ]:
# Define initial model

model0 = Model(vp=v0, origin=origin, shape=shape, spacing=spacing,
              space_order=2, nbl=10, dt=2.449, bcs="damp")

In [ ]:
# Define acquisition geometry: source

t0 = 0.
tn = 1000.
f0 = 0.010
# First, position source centrally in all dimensions, then set depth
src_coordinates = np.empty((1, 2))
src_coordinates[0, :] = np.array(model0.domain_size) * .5
src_coordinates[0, 0] = 20.  # Depth is 20m

# Define acquisition geometry: receivers

# Initialize receivers for synthetic and imaging data
rec_coordinates = np.empty((nreceivers, 2))
rec_coordinates[:, 1] = np.linspace(0, model0.domain_size[0], num=nreceivers)
rec_coordinates[:, 0] = 980.

# Geometry

geometry = AcquisitionGeometry(model0, rec_coordinates, src_coordinates, t0, tn, f0=f0, src_type='Ricker')
# We can plot the time signature to see the wavelet
geometry.src.show()

In [ ]:
# Plot acquisition geometry with initial guess of the velocity model
plot_velocity(model0, source=geometry.src_positions,
              receiver=geometry.rec_positions[::4, :])

In [ ]:
# Create solver

solver = AcousticWaveSolver(model0, geometry, space_order=4)

In [ ]:
# Prepare the varying source locations sources
source_locations = np.empty((nshots, 2), dtype=np.float32)
source_locations[:, 0] = 30.
source_locations[:, 1] = np.linspace(0., 1000, num=nshots)

## For part 1 you will need to edit the cell below

In [ ]:
# Computes the residual between observed and synthetic data into the residual
def compute_residual(residual, dobs, dsyn):
    residual = None
    return residual

## For part 1 you will need to edit the cell below

In [ ]:
# Create FWI gradient kernel

def fwi_gradient(vp_in):    
    # Create symbols to hold the gradient
    grad = Function(name="grad", grid=model0.grid)
    # Create placeholders for the data residual and data
    residual = Receiver(name='residual', grid=model0.grid,
                        time_range=geometry.time_axis, 
                        coordinates=geometry.rec_positions)
    d_obs = Receiver(name='d_obs', grid=model0.grid,
                     time_range=geometry.time_axis, 
                     coordinates=geometry.rec_positions)
    d_syn = Receiver(name='d_syn', grid=model0.grid,
                     time_range=geometry.time_axis, 
                     coordinates=geometry.rec_positions)
    objective = 0.
    for i in range(nshots):
        # Update source location
        geometry.src_positions[0, :] = source_locations[i, :]
        
        # Load synthetic data from file
        ...
        d_obs = None
        
        # Compute smooth data and full forward wavefield u0
        _, u0, _ = solver.forward(vp=vp_in, save=True, rec=d_syn)
        
        # Compute gradient from data residual and update objective function 
        compute_residual(residual, d_obs, d_syn)
        
        objective += .5*norm(residual)**2
        solver.gradient(rec=residual, u=u0, vp=vp_in, grad=grad)
    
    return objective, grad

In [ ]:
from sympy import Min, Max
# Define bounding box constraints on the solution.
def update_with_box(vp, alpha, dm, vmin=2.0, vmax=3.5):
    """
    Apply gradient update in-place to vp with box constraint

    Notes:
    ------
    For more advanced algorithm, one will need to gather the non-distributed
    velocity array to apply constrains and such.
    """
    update = vp + alpha * dm
    update_eq = Eq(vp, Max(Min(update, vmax), vmin))
    Operator(update_eq)()

## In part 1 you do not need to edit the cell below.
## In part 2 you will need to modify the manner in which alpha is determined in the cell below.

In [ ]:
# Run FWI with gradient descent
history = np.zeros((fwi_iterations, 1))
for i in range(0, fwi_iterations):
    # Compute the functional value and gradient for the current
    # model estimate
    phi, direction = fwi_gradient(model0.vp)

    # Store the history of the functional values
    history[i] = phi

    # In part 2 you will need to change how this alpha is chosen
    alpha = .05 / mmax(direction)

    # Update the model estimate and enforce minimum/maximum values
    update_with_box(model0.vp , alpha , direction)

    # Log the progress made
    print('Objective value is %f at iteration %d' % (phi, i+1))

In [ ]:
# Plot inverted velocity model
plot_velocity(model0)

In [ ]:
# Plot objective function decrease
plt.figure()
plt.loglog(history)
plt.xlabel('Iteration number')
plt.ylabel('Misift value Phi')
plt.title('Convergence')
plt.show()

## Part 2(i)

Add text here.

## Part 2(ii)

Add text here.